# Multilayer Perceptron from raw data
This notebook will guide you through the use of the `keras` package to train a multilayer perceptron for handwritten digits classification. You are going to use the `mnist` dataset from LeCun et al. 1998

## Loading the packages

In [9]:
%pip install tensorflow --upgrade
%pip install keras --upgrade

import numpy as np
from matplotlib import pyplot as pl

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.utils import np_utils
from sklearn import metrics as me

%matplotlib inline

Note: you may need to restart the kernel to use updated packages.



## Using raw data to train a MLP
First load the `mnist` dataset and normalize it to be in the range [0, 1]

In [10]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

n_classes = 10
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

60000 train samples
10000 test samples


Create the MLP

784 entrées 10 sortie et 300 neurones dans la couche cachée.

In [11]:
model = Sequential()
model.add(Dense(300, input_shape=(784,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 300)               235500    
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                3010      
                                                                 
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________


Define some constants and train the MLP

In [ ]:
#copy for test 
batch_size = 10, 30, 50, 70, 90,110, 128
n_epoch = 5, 10, 15,20,25,30,35

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

for i in batch_size:
    for j in n_epoch:
        history = model.fit(X_train, Y_train,
                    batch_size=i, epochs=j,
                    verbose=1, validation_data=(X_test, Y_test))

Epoch 1/5
6000/6000 [==============================] - 13s 2ms/step - loss: 0.3251 - accuracy: 0.9110 - val_loss: 0.1710 - val_accuracy: 0.9562
Epoch 2/5
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2383 - accuracy: 0.9473 - val_loss: 0.1481 - val_accuracy: 0.9674
Epoch 3/5
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2291 - accuracy: 0.9550 - val_loss: 0.1570 - val_accuracy: 0.9702
Epoch 4/5
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2332 - accuracy: 0.9580 - val_loss: 0.1619 - val_accuracy: 0.9712
Epoch 5/5
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2334 - accuracy: 0.9615 - val_loss: 0.1552 - val_accuracy: 0.9753
Epoch 1/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2303 - accuracy: 0.9626 - val_loss: 0.1730 - val_accuracy: 0.9734
Epoch 2/10
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2283 - accuracy: 0.9646 - val_loss: 0.1755 - val_accurac

6000/6000 [==============================] - 13s 2ms/step - loss: 0.1992 - accuracy: 0.9795 - val_loss: 0.3108 - val_accuracy: 0.9776
Epoch 8/25
6000/6000 [==============================] - 13s 2ms/step - loss: 0.1974 - accuracy: 0.9796 - val_loss: 0.3094 - val_accuracy: 0.9778
Epoch 9/25
6000/6000 [==============================] - 13s 2ms/step - loss: 0.2058 - accuracy: 0.9794 - val_loss: 0.3031 - val_accuracy: 0.9791
Epoch 10/25
6000/6000 [==============================] - 13s 2ms/step - loss: 0.2013 - accuracy: 0.9802 - val_loss: 0.3252 - val_accuracy: 0.9795
Epoch 11/25
6000/6000 [==============================] - 12s 2ms/step - loss: 0.1877 - accuracy: 0.9797 - val_loss: 0.3098 - val_accuracy: 0.9802
Epoch 12/25
6000/6000 [==============================] - 12s 2ms/step - loss: 0.1932 - accuracy: 0.9798 - val_loss: 0.3345 - val_accuracy: 0.9782
Epoch 13/25
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2072 - accuracy: 0.9801 - val_loss: 0.3029 - val_accuracy:

6000/6000 [==============================] - 14s 2ms/step - loss: 0.1777 - accuracy: 0.9839 - val_loss: 0.3925 - val_accuracy: 0.9792
Epoch 9/35
6000/6000 [==============================] - 14s 2ms/step - loss: 0.1740 - accuracy: 0.9843 - val_loss: 0.4001 - val_accuracy: 0.9772
Epoch 10/35
6000/6000 [==============================] - 13s 2ms/step - loss: 0.1694 - accuracy: 0.9839 - val_loss: 0.3872 - val_accuracy: 0.9791
Epoch 11/35
6000/6000 [==============================] - 13s 2ms/step - loss: 0.1696 - accuracy: 0.9847 - val_loss: 0.3991 - val_accuracy: 0.9797
Epoch 12/35
6000/6000 [==============================] - 13s 2ms/step - loss: 0.1755 - accuracy: 0.9837 - val_loss: 0.3847 - val_accuracy: 0.9782
Epoch 13/35
6000/6000 [==============================] - 13s 2ms/step - loss: 0.1747 - accuracy: 0.9834 - val_loss: 0.3890 - val_accuracy: 0.9783
Epoch 14/35
6000/6000 [==============================] - 13s 2ms/step - loss: 0.1722 - accuracy: 0.9844 - val_loss: 0.3974 - val_accuracy

2000/2000 [==============================] - 6s 3ms/step - loss: 0.1096 - accuracy: 0.9862 - val_loss: 0.4004 - val_accuracy: 0.9768
Epoch 15/15
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1231 - accuracy: 0.9860 - val_loss: 0.3723 - val_accuracy: 0.9783
Epoch 1/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1206 - accuracy: 0.9863 - val_loss: 0.4124 - val_accuracy: 0.9762
Epoch 2/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1139 - accuracy: 0.9868 - val_loss: 0.4025 - val_accuracy: 0.9767
Epoch 3/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1175 - accuracy: 0.9855 - val_loss: 0.4011 - val_accuracy: 0.9762
Epoch 4/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1231 - accuracy: 0.9860 - val_loss: 0.3885 - val_accuracy: 0.9779
Epoch 5/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1174 - accuracy: 0.9860 - val_loss: 0.3797 - val_accuracy: 0.9773
Ep

Epoch 11/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1054 - accuracy: 0.9875 - val_loss: 0.4575 - val_accuracy: 0.9763
Epoch 12/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1113 - accuracy: 0.9879 - val_loss: 0.4376 - val_accuracy: 0.9776
Epoch 13/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1077 - accuracy: 0.9876 - val_loss: 0.4135 - val_accuracy: 0.9778
Epoch 14/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1114 - accuracy: 0.9880 - val_loss: 0.4158 - val_accuracy: 0.9781
Epoch 15/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1042 - accuracy: 0.9890 - val_loss: 0.4600 - val_accuracy: 0.9769
Epoch 16/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1061 - accuracy: 0.9881 - val_loss: 0.4362 - val_accuracy: 0.9772
Epoch 17/30
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1171 - accuracy: 0.9878 - val_loss: 0.4306 - val_ac

1200/1200 [==============================] - 5s 4ms/step - loss: 0.0918 - accuracy: 0.9895 - val_loss: 0.4737 - val_accuracy: 0.9763
Epoch 3/5
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0965 - accuracy: 0.9891 - val_loss: 0.4510 - val_accuracy: 0.9777
Epoch 4/5
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0928 - accuracy: 0.9894 - val_loss: 0.4609 - val_accuracy: 0.9770
Epoch 5/5
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0922 - accuracy: 0.9891 - val_loss: 0.4682 - val_accuracy: 0.9766
Epoch 1/10
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0905 - accuracy: 0.9898 - val_loss: 0.4675 - val_accuracy: 0.9763
Epoch 2/10
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0895 - accuracy: 0.9896 - val_loss: 0.4711 - val_accuracy: 0.9767
Epoch 3/10
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0946 - accuracy: 0.9891 - val_loss: 0.4544 - val_accuracy: 0.9765
Epoch 

1200/1200 [==============================] - 5s 4ms/step - loss: 0.0898 - accuracy: 0.9897 - val_loss: 0.5112 - val_accuracy: 0.9758
Epoch 9/25
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0855 - accuracy: 0.9902 - val_loss: 0.4914 - val_accuracy: 0.9764
Epoch 10/25
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0951 - accuracy: 0.9894 - val_loss: 0.5067 - val_accuracy: 0.9754
Epoch 11/25
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0976 - accuracy: 0.9901 - val_loss: 0.4993 - val_accuracy: 0.9759
Epoch 12/25
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0941 - accuracy: 0.9903 - val_loss: 0.4942 - val_accuracy: 0.9767
Epoch 13/25
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0880 - accuracy: 0.9901 - val_loss: 0.5077 - val_accuracy: 0.9762
Epoch 14/25
1200/1200 [==============================] - 5s 4ms/step - loss: 0.0788 - accuracy: 0.9906 - val_loss: 0.5211 - val_accuracy: 0.974

In [ ]:
batch_size = 128
n_epoch = 10

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, epochs=n_epoch,
                    verbose=1, validation_data=(X_test, Y_test))

Show the performance of the model

In [ ]:
pl.plot(history.history['loss'], label='Training')
pl.plot(history.history['val_loss'], label='Testing')
pl.legend()
pl.grid()

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Confusion matrix

In [ ]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=-1)
me.confusion_matrix(y_test, pred)